In [1]:
import numpy as np
import pandas as pd

kospi200 = np.load('kospi200.npy')
samsung = np.load('samsung.npy')

print(kospi200)
print(samsung)
print(kospi200.shape)
print(samsung.shape)

[[1.000e+00 0.000e+00]
 [2.000e+00 0.000e+00]
 [3.000e+00 0.000e+00]
 ...
 [1.094e+03 2.000e-01]
 [1.095e+03 0.000e+00]
 [1.096e+03 0.000e+00]]
[[1.000e+00 9.063e+00]
 [2.000e+00 9.104e+00]
 [3.000e+00 3.198e+00]
 ...
 [1.094e+03 6.961e+00]
 [1.095e+03 6.997e+00]
 [1.096e+03 7.022e+00]]
(1096, 2)
(1096, 2)


In [4]:
def split_xy5(dataset, time_steps, y_column):
    x, y = list(), list()
    for i in range(len(dataset)):
        x_end_number = i + time_steps
        y_end_number = x_end_number + y_column # 수정

        if y_end_number > len(dataset):  # 수정
            break
        tmp_x = dataset[i:x_end_number, :]  # 수정
        tmp_y = dataset[x_end_number:y_end_number, 1]    # 수정
        x.append(tmp_x)
        y.append(tmp_y)
    return np.array(x), np.array(y)
x, y = split_xy5(samsung, 5, 1) 
print(x[0,:], "\n", y[0])
print(x.shape)
print(y.shape)
# 데이터 셋 나누기
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score
x_train, x_test, y_train, y_test = train_test_split(
    x, y, random_state=1, test_size = 0.3)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

x_train = np.reshape(x_train,
    (x_train.shape[0], x_train.shape[1] * x_train.shape[2]))
x_test = np.reshape(x_test,
    (x_test.shape[0], x_test.shape[1] * x_test.shape[2]))
print(x_train.shape)
print(x_test.shape)

#### 데이터 전처리 #####
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)
print(x_train_scaled[0, :])

[[ 1.     9.063]
 [ 2.     9.104]
 [ 3.     3.198]
 [ 4.     3.493]
 [ 5.    14.994]] 
 [3.202]
(1091, 5, 2)
(1091, 1)
(763, 5, 2)
(328, 5, 2)
(763, 1)
(328, 1)
(763, 10)
(328, 10)
[0.18179804 0.14462999 0.18179804 0.17092885 0.18179804 0.1845877
 0.18179804 0.27908513 0.18179804 0.30779219]


In [5]:
x_train_scaled = np.reshape(x_train_scaled,
    (x_train_scaled.shape[0], 5, 2))
x_test_scaled = np.reshape(x_test_scaled,
    (x_test_scaled.shape[0], 5, 2))
print(x_train_scaled.shape)
print(x_test_scaled.shape)

(763, 5, 2)
(328, 5, 2)


In [7]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

# 모델구성
model = Sequential()
model.add(LSTM(64, input_shape=(5, 2)))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

In [8]:
model.compile(loss='mse', optimizer='adam', metrics=['mse'])

from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=20)
model.fit(x_train_scaled, y_train, validation_split=0.2, verbose=1,
          batch_size=1, epochs=100, callbacks=[early_stopping])

loss, mse = model.evaluate(x_test_scaled, y_test, batch_size=1)
print('loss : ', loss)
print('mse : ', mse)

Epoch 1/100
610/610 [==============================] - 2s 2ms/step - loss: 21752.7480 - mse: 21752.7480 - val_loss: 55047.5508 - val_mse: 55047.5508
Epoch 2/100
610/610 [==============================] - 1s 2ms/step - loss: 17581.9453 - mse: 17581.9453 - val_loss: 56537.6797 - val_mse: 56537.6797
Epoch 3/100
610/610 [==============================] - 1s 2ms/step - loss: 15839.4248 - mse: 15839.4248 - val_loss: 53852.6016 - val_mse: 53852.6016
Epoch 4/100
610/610 [==============================] - 1s 2ms/step - loss: 12611.7910 - mse: 12611.7910 - val_loss: 56994.2031 - val_mse: 56994.2031
Epoch 5/100
610/610 [==============================] - 1s 2ms/step - loss: 12694.5586 - mse: 12694.5586 - val_loss: 51676.6094 - val_mse: 51676.6094
Epoch 6/100
610/610 [==============================] - 1s 2ms/step - loss: 11935.8457 - mse: 11935.8457 - val_loss: 52352.8672 - val_mse: 52352.8672
Epoch 7/100
610/610 [==============================] - 1s 2ms/step - loss: 10134.1426 - mse: 10134.1426 - 

610/610 [==============================] - 1s 2ms/step - loss: 3964.0669 - mse: 3964.0669 - val_loss: 50966.0859 - val_mse: 50966.0859
Epoch 57/100
610/610 [==============================] - 1s 2ms/step - loss: 5045.5259 - mse: 5045.5259 - val_loss: 45580.0625 - val_mse: 45580.0625
Epoch 58/100
610/610 [==============================] - 1s 2ms/step - loss: 4639.7427 - mse: 4639.7427 - val_loss: 55052.7383 - val_mse: 55052.7383
Epoch 59/100
610/610 [==============================] - 1s 2ms/step - loss: 4405.5015 - mse: 4405.5015 - val_loss: 48890.5664 - val_mse: 48890.5664
Epoch 60/100
610/610 [==============================] - 1s 2ms/step - loss: 3483.8093 - mse: 3483.8093 - val_loss: 48576.8320 - val_mse: 48576.8320
Epoch 61/100
610/610 [==============================] - 1s 2ms/step - loss: 4588.1294 - mse: 4588.1294 - val_loss: 54484.7852 - val_mse: 54484.7852
Epoch 62/100
610/610 [==============================] - 1s 2ms/step - loss: 4872.6382 - mse: 4872.6382 - val_loss: 57819.0312

In [10]:
y_pred = model.predict(x_test_scaled)

for i in range(5):
    print('종가 : ', y_test[i], '/ 예측가 : ', y_pred[i])



11/11 [==============================] - 0s 1ms/step
종가 :  [13.273] / 예측가 :  [19.76843]
종가 :  [1.446] / 예측가 :  [13.537819]
종가 :  [986.241] / 예측가 :  [749.7589]
종가 :  [23.789] / 예측가 :  [12.730897]
종가 :  [18.329] / 예측가 :  [16.87341]
